Consider the evolution of a simple Hamiltonian into a problem-specific Hamiltonian where the optimal solution is 'encoded' in the ground state of the problem Hamiltonian $\displaystyle H_{\text{P}}$

$$H\left(s\right) = \left(1 - s\right)H_0 + sH_{\text{P}}$$

where

$$H_{\text{P}} = \underbrace{A\left[\sum_{j = 0}^{\ell}2^jy_j + \left(c + 1 - 2^{\ell + 1}\right)y_{\ell + 1} - \sum_{i = 1}^nw_ix_i\right]^2}_{H_A} + \underbrace{\left(-B\sum_{i = 1}^np_ix_i\right)}_{H_B}$$

for $\displaystyle s = \frac{t}{T}$ where $T$ is the total computation time and $\displaystyle \ell = \lfloor\log_2\lparen c - 1\rparen\rfloor$


In [1]:
from qutip import tensor, basis, sigmaz, qeye, mesolve
import numpy as np